In [11]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os
import matplotlib.pyplot as plt
import numpy as np

In [12]:
mnist = input_data.read_data_sets(os.path.join('.', 'mnist'), one_hot = True)
tf.logging.set_verbosity(1)

Extracting .\mnist\train-images-idx3-ubyte.gz
Extracting .\mnist\train-labels-idx1-ubyte.gz
Extracting .\mnist\t10k-images-idx3-ubyte.gz
Extracting .\mnist\t10k-labels-idx1-ubyte.gz


In [13]:
train_x, train_y, test_x, test_y = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

In [48]:
def mlp(x, n_inputs, n_outputs, n_layers, n_neurons):
    weights = []
    biases = []
    for i in range(n_layers):
        
        #weights
        curr_weight = tf.Variable(tf.random_normal([n_inputs if i == 0 else n_neurons[i-1], n_neurons[i]]), 
                                  name = "w_" + str(i))
        weights.append(curr_weight)
        
        #bias
        curr_bias = tf.Variable(tf.random_normal([n_neurons[i]]), name = "b_" + str(i))
        biases.append(curr_bias)
        
    # For the last layer
    
    #weight
    curr_weight = tf.Variable(tf.random_normal([n_neurons[n_layers - 1] if n_layers > 0 else n_inputs, n_outputs]),
                             name = "w_out")
    weights.append(curr_weight)
    
    #bias
    curr_bias = tf.Variable(tf.random_normal([n_outputs]), name = "b_out")
    biases.append(curr_bias)
    
    # start
    layer = x
    
    # for the hidden layers
    for i in range(n_layers):
        layer = tf.nn.relu(tf.matmul(layer, weights[i]) + biases[i])
        print(layer.shape, weights[i].shape, biases[i].shape)
        
    # for the output layer
    print(layer.shape, weights[n_layers].shape, biases[n_layers].shape)
    layer = tf.matmul(layer, weights[n_layers]) + biases[n_layers]
    
    return layer

In [49]:
def mnist_batches(batch_size = 100):
    X_batch, Y_batch = mnist.train.next_batch(batch_size)  # sending the training data through here
    return [X_batch, Y_batch]

In [50]:
def tensorflow_classification(batch_function, batch_size, n_batches,
                              n_epochs, model, loss, optimizer, accuracy_function,
                              x_test, y_test):

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(n_epochs):
            epoch_loss = 0.0
            for batch in range(n_batches):
                X_batch, Y_batch = batch_function(batch_size)
                a, batch_loss = sess.run([optimizer, loss], feed_dict = { x : X_batch, y : Y_batch })
                epoch_loss += batch_loss
            avg_loss = epoch_loss / n_batches
            print("For epoch : " + str(epoch))
            print("a :" + str(a))
            print("loss : " + str(avg_loss))
        
        # Now for our testset
        
        accuracy_score = sess.run(accuracy_function, feed_dict = { x : x_test, y : y_test })
        print("Accuracy : " + str(accuracy_score))

In [51]:
tf.reset_default_graph()

n_layers = 2
n_neurons = []

for i in range(n_layers):
    n_neurons.append(256)

n_epochs = 5
batch_size = 100
learning_rate = 0.01
n_batches = mnist.train.num_examples / batch_size

n_inputs = 784
n_outputs = 10

In [52]:
# input images

x = tf.placeholder(dtype="float32", shape = [None, n_inputs], name = "x")
y = tf.placeholder(dtype="float32", shape = [None, n_outputs], name = "y")

# set the model

model = mlp(x, n_inputs, n_outputs, n_layers, n_neurons)

# calculate the loss

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = model, labels = y))

# optimizer

optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

predictions_check = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
accuracy_function = tf.reduce_mean(tf.cast(predictions_check, tf.float32))

tensorflow_classification(mnist_batches, batch_size, int(n_batches),
                              n_epochs, model, loss, optimizer,
                              accuracy_function, test_x, test_y)

(?, 256) (784, 256) (256,)
(?, 256) (256, 256) (256,)
(?, 256) (256, 10) (10,)
Level 1:tensorflow:Gradient for 'Mean'
Level 1:tensorflow:  in  --> gradients/Fill:0
Level 1:tensorflow:  out --> gradients/Mean_grad/truediv:0
Level 1:tensorflow:Gradient for 'softmax_cross_entropy_with_logits/Reshape_2'
Level 1:tensorflow:  in  --> gradients/Mean_grad/truediv:0
Level 1:tensorflow:  out --> gradients/softmax_cross_entropy_with_logits/Reshape_2_grad/Reshape:0
Level 1:tensorflow:Gradient for 'softmax_cross_entropy_with_logits'
Level 1:tensorflow:  in  --> gradients/softmax_cross_entropy_with_logits/Reshape_2_grad/Reshape:0, gradients/zeros_like:0
Level 1:tensorflow:  out --> gradients/softmax_cross_entropy_with_logits_grad/tuple/control_dependency:0, gradients/softmax_cross_entropy_with_logits_grad/tuple/control_dependency_1:0
Level 1:tensorflow:Gradient for 'softmax_cross_entropy_with_logits/Reshape'
Level 1:tensorflow:  in  --> gradients/softmax_cross_entropy_with_logits_grad/tuple/control_